# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [2]:
url = "https://books.toscrape.com/catalogue/page-1.html"
response = requests.get(url)

soup = BeautifulSoup(response.content)

#identify the grid
grid = soup.find("ol", attrs = {"class":"row"})

#within the grid, identify all books
books = grid.find_all("article", attrs = {"class":"product_pod"})

In [3]:
#title
books[0].find_all("a")[-1]["title"]

#price
books[0].find("p", attrs = {"class":"price_color"}).get_text()

#rating
books[0].find("p", attrs = {"class":"star-rating"})["class"][-1]

'Three'

In [4]:
def get_title(book):
    return book.find_all("a")[-1]["title"]

def get_price(book):
    raw_price = book.find("p", attrs = {"class":"price_color"}).get_text()
    clean_price = float(raw_price.replace("£", ""))
    return float(clean_price)

def get_rating(book):
    raw_rating = book.find("p", attrs = {"class":"star-rating"})["class"][-1]
    if raw_rating == "One":
        return 1
    elif raw_rating == "Two":
        return 2
    elif raw_rating == "Three":
        return 3
    elif raw_rating == "Four":
        return 4
    else:
        return 5
    
def get_link(book):
    domain = "https://books.toscrape.com/catalogue/"
    href = book.find_all("a")[-1]["href"]

    return domain + href


def get_upc(soup_book):
    upc = soup_book.find("td").get_text()
    return upc


def get_availability(soup_book):
    return soup_book.find("p", attrs={"class":"instock availability"}).get_text().strip()

def get_description(soup_book):
    return soup_book.find("div", id="product_description").find_next("p").get_text().strip()

def get_genre(soup_book):
    return soup_book.find("ul", attrs = {"class":"breadcrumb"}).find_all("li")[-2].get_text().strip()

In [5]:
def scrape_books(max_price, min_rating, page_number):

    url = f"https://books.toscrape.com/catalogue/page-{page_number}.html"
    response = requests.get(url)

    soup = BeautifulSoup(response.content)

    #identify the grid
    grid = soup.find("ol", attrs = {"class":"row"})

    #within the grid, identify all books
    books = grid.find_all("article", attrs = {"class":"product_pod"})

    dict = {}
    index = 0

    for book in books:
        title = get_title(book)
        price = get_price(book)
        rating = get_rating(book)
        book_url = get_link(book)



        if max_price >= price and min_rating <= rating:
            #if both conditions above are met, we need to extract more data (UPC, Genre, etc)
            r_book = requests.get(book_url)
            soup_book = BeautifulSoup(r_book.content)

            #extra fields to scrap
            upc = get_upc(soup_book)
            availability = get_availability(soup_book)
            description = get_description(soup_book)
            genre = get_genre(soup_book)


            dict[index] = {"title": title,
                        "price":price,
                        "rating":rating,
                        "url": book_url,
                        "upc":upc,
                        "availability": availability,
                        "description":description,
                        "genre": genre}
            
            index +=1
        else:
            pass

    return pd.DataFrame.from_dict(dict, orient = "index")

In [ ]:
max_price = 20
min_rating = 4

list_of_dfs = []
for i in range(1, 51):
   df = scrape_books(max_price= max_price, 
                     min_rating= min_rating, 
                     page_number=i)
   
   list_of_dfs.append(df)
   print(f"Scraping page number {i}")

In [ ]:
full_df = pd.concat(list_of_dfs, ignore_index=True)
full_df

In [8]:
#alternative -> for loop inside the function

def scrape_books(max_price, min_rating):

    list_of_dfs = []
    for page_number in range(1, 51):

        url = f"https://books.toscrape.com/catalogue/page-{page_number}.html"
        response = requests.get(url)

        soup = BeautifulSoup(response.content)

        #identify the grid
        grid = soup.find("ol", attrs = {"class":"row"})

        #within the grid, identify all books
        books = grid.find_all("article", attrs = {"class":"product_pod"})

        dict = {}
        index = 0

        for book in books:
            title = get_title(book)
            price = get_price(book)
            rating = get_rating(book)
            book_url = get_link(book)



            if max_price >= price and min_rating <= rating:
                #if both conditions above are met, we need to extract more data (UPC, Genre, etc)
                r_book = requests.get(book_url)
                soup_book = BeautifulSoup(r_book.content)

                #extra fields to scrap
                upc = get_upc(soup_book)
                availability = get_availability(soup_book)
                description = get_description(soup_book)
                genre = get_genre(soup_book)


                dict[index] = {"title": title,
                            "price":price,
                            "rating":rating,
                            "url": book_url,
                            "upc":upc,
                            "availability": availability,
                            "description":description,
                            "genre": genre}
                
                index +=1
            else:
                pass

        page_df = pd.DataFrame.from_dict(dict, orient = "index")
        list_of_dfs.append(page_df)

    return pd.concat(list_of_dfs, ignore_index=True)


In [ ]:
scrape_books(20, 4)